In [6]:
import open3d as o3d
import numpy as np
from collections import deque


In [7]:
%cd /Users/deeprodge/Downloads/DREAMS/PG&E/rock_detection_3d/unsupervised_rock_detection_3d

/Users/deeprodge/Downloads/DREAMS/PG&E/rock_detection_3d/unsupervised_rock_detection_3d


In [1]:
import argparse
import numpy as np
import open3d as o3d
from collections import deque
import time

class RegionGrowingSegmentation:
    def __init__(self, pcd, voxel_size=0.01, num_neighbors=50, distance_threshold=0.05):
        self.pcd = pcd.voxel_down_sample(voxel_size)
        
        # Estimate normals
        radius_normal = voxel_size * 5
        self.pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
        self.pcd.orient_normals_consistent_tangent_plane(k=10)
        self.pcd.normals = o3d.utility.Vector3dVector(-np.asarray(self.pcd.normals))
        
        self.num_neighbors = num_neighbors
        self.distance_threshold = distance_threshold
        
        self.labels = np.array([-1] * len(self.pcd.points))  # -1 indicates unlabeled
        self.normals = np.asarray(self.pcd.normals)
        self.pcd_tree = o3d.geometry.KDTreeFlann(self.pcd)
        
        # Precompute local statistics for adaptive thresholds
        self.local_mean_dot_product, self.local_std_dot_product = self.compute_local_statistics()
        self.local_mean_curvature, self.local_std_curvature = self.compute_local_curvature_statistics()

    def precompute_neighbors(self):
        neighbors = []
        for i in range(len(self.pcd.points)):
            [k, idx, _] = self.pcd_tree.search_radius_vector_3d(self.pcd.points[i], self.distance_threshold)
            neighbors.append(np.array(idx[1:]))  # Skip the first index as it's the point itself
        return neighbors

    def compute_local_statistics(self):
        mean_dot_products = np.zeros(len(self.pcd.points))
        std_dot_products = np.zeros(len(self.pcd.points))
        
        for i in range(len(self.pcd.points)):
            neighbors = self.pcd_tree.search_radius_vector_3d(self.pcd.points[i], self.distance_threshold)[1]
            neighbor_normals = self.normals[neighbors]
            dot_products = np.clip(np.dot(neighbor_normals, self.normals[i]), -1.0, 1.0)
            mean_dot_products[i] = np.mean(dot_products)
            std_dot_products[i] = np.std(dot_products)
        
        return mean_dot_products, std_dot_products

    def compute_local_curvature_statistics(self):
        mean_curvature = np.zeros(len(self.pcd.points))
        std_curvature = np.zeros(len(self.pcd.points))
        
        for i in range(len(self.pcd.points)):
            curvatures = [self.estimate_curvature(n) for n in self.pcd_tree.search_radius_vector_3d(self.pcd.points[i], self.distance_threshold)[1]]
            mean_curvature[i] = np.mean(curvatures)
            std_curvature[i] = np.std(curvatures)
        
        return mean_curvature, std_curvature

    def estimate_curvature(self, index):
        k, idx, _ = self.pcd_tree.search_radius_vector_3d(self.pcd.points[index], self.distance_threshold)
        if k > 1:
            neighbor_normals = self.normals[idx, :]
            curvature = np.mean(np.linalg.norm(np.cross(neighbor_normals - self.normals[index], neighbor_normals), axis=1))
            return curvature
        else:
            return 0

    def adaptive_thresholds(self, index):
        mean_dot = self.local_mean_dot_product[index]
        std_dot = self.local_std_dot_product[index]
        mean_curv = self.local_mean_curvature[index]
        std_curv = self.local_std_curvature[index]
        
        smoothness_threshold = mean_dot - std_dot  # Example of setting adaptive threshold
        curvature_threshold = mean_curv + std_curv  # Example of setting adaptive threshold

        print(smoothness_threshold, curvature_threshold)

        
        return smoothness_threshold, curvature_threshold

    def grow_region(self, starting_index, region_index, neighbors):
        queue = deque([starting_index])
        self.labels[starting_index] = region_index

        while queue:
            current_index = queue.popleft()
            current_normal = self.normals[current_index]

            neighbor_indices = neighbors[current_index]

            for neighbor_index in neighbor_indices:
                if self.labels[neighbor_index] != -1:
                    continue

                # Calculate adaptive thresholds for the neighbor
                smoothness_threshold, curvature_threshold = self.adaptive_thresholds(neighbor_index)
                
                # Calculate segmentation criteria for the neighbor
                min_dot_product = self.calculate_segmentation_criteria(neighbor_index, neighbors)

                # Compute curvature for the neighbor
                curvature = self.estimate_curvature(neighbor_index)

                # Apply the adaptive thresholds
                if min_dot_product >= smoothness_threshold and curvature < curvature_threshold:
                    self.labels[neighbor_index] = region_index
                    queue.append(neighbor_index)

    def calculate_segmentation_criteria(self, neighbor_index, neighbors):
        neighbor_normal = self.normals[neighbor_index]
        second_order_neighbors = neighbors[neighbor_index]

        # Filter second-order neighbors to keep only those already segmented as PBR
        filtered_neighbors = [n for n in second_order_neighbors if self.labels[n] != -1]
        if len(filtered_neighbors) == 0:
            return float('inf')

        filtered_normals = self.normals[filtered_neighbors]
        dot_products = np.clip(np.dot(filtered_normals, neighbor_normal), -1.0, 1.0)

        return np.min(dot_products)

    def segment(self):
        # Compute the bounding box of the point cloud
        points = np.asarray(self.pcd.points)
        min_bound = points.min(axis=0)
        max_bound = points.max(axis=0)

        # Compute the centroid of the bounding box in the x and y dimensions
        centroid_x = (min_bound[0] + max_bound[0]) / 2.0
        centroid_y = (min_bound[1] + max_bound[1]) / 2.0

        # Find the point with the highest z value near the centroid in x and y dimensions
        distances = np.linalg.norm(points[:, :2] - np.array([centroid_x, centroid_y]), axis=1)
        highest_point_index = np.argmax(points[:, 2] - distances)

        # Find the bottommost corner of the bounding box
        bottommost_point_index = np.argmin(points[:, 2])

        neighbors = self.precompute_neighbors()
        
        # Grow region from the highest point near the centroid (region 0)
        self.grow_region(highest_point_index, region_index=0, neighbors=neighbors)
        
        # Grow region from the bottommost point (region 1)
        self.grow_region(bottommost_point_index, region_index=1, neighbors=neighbors)

        return self.pcd, self.labels

    def color_point_cloud(self):
        points = np.asarray(self.pcd.points)
        print(len(points), "points")
        colors = np.zeros_like(points)

        # Color all points grey
        colors[:, :] = [0.5, 0.5, 0.5]  # Grey color

        # Color the labeled points
        colors[self.labels == 0] = [1, 0, 0]  # Red color for region 0
        colors[self.labels == 1] = [0, 0, 1]  # Blue color for region 1

        # Create a new point cloud with the updated colors
        colored_pcd = o3d.geometry.PointCloud()
        colored_pcd.points = o3d.utility.Vector3dVector(points)
        colored_pcd.colors = o3d.utility.Vector3dVector(colors)

        self.pcd.colors = o3d.utility.Vector3dVector(colors)

        return self.pcd

    def conditional_label_propagation(self, distance_threshold=0.05):
        points = np.asarray(self.pcd.points)
        tree = self.pcd_tree
        
        unlabeled_indices = np.where(self.labels == -1)[0]
        
        for index in unlabeled_indices:
            [k, idx, _] = tree.search_radius_vector_3d(points[index], distance_threshold)
            neighbor_labels = self.labels[idx]
        
            # Exclude unlabeled neighbors
            labeled_neighbors = neighbor_labels[neighbor_labels != -1]
            if len(labeled_neighbors) > 0:
                # Assign the majority label among the neighbors
                self.labels[index] = np.bincount(labeled_neighbors).argmax()
        
        return self.labels



In [4]:
%%time

# Load point cloud

#67
# pcd = o3d.io.read_point_cloud("box_pbr/pbr67.pcd")
# segmenter = RegionGrowingSegmentation(pcd,
#                                       distance_threshold=0.05, 
#                                       smoothness_threshold=0.96, # Decreasing it makes growing Aggressive (lenient threshold)
#                                       curvature_threshold = 0.1, # Decreasing it makes growing conservative (Strict threshold)
#                                       #use_curvature=False,
#                                      )

#28
pcd = o3d.io.read_point_cloud("box_pbr/pbr28.pcd")
segmenter = RegionGrowingSegmentation(pcd,
                                      distance_threshold=0.05,
                                      smoothness_threshold=0.98,
                                      curvature_threshold = 0.15,
                                      #use_curvature=False,
                                     )

# #42
# pcd = o3d.io.read_point_cloud("box_pbr/pbr42.pcd")
# segmenter = RegionGrowingSegmentation(pcd,
#                                       distance_threshold=0.05,
#                                       smoothness_threshold=0.98,
#                                       curvature_threshold = 0.1,
#                                       #use_smoothness=False,
#                                      )
pcd, labels = segmenter.segment()

# Apply morphological closing to the labels
#segmenter.morphological_closing(structure_size=1000)
segmenter.conditional_label_propagation(distance_threshold=0.05)

# Color the point cloud
colored_pcd = segmenter.color_point_cloud()


CPU times: user 13.8 s, sys: 84.7 ms, total: 13.9 s
Wall time: 13.8 s


In [5]:
# Visualize the point cloud
o3d.visualization.draw_geometries([colored_pcd],
                                  #point_show_normal = True,
                                 )

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [100]:
import numpy as np
import open3d as o3d
from collections import deque

class RegionGrowingSegmentation:
    def __init__(self, pcd, num_neighbors=50, smoothness_threshold=0.5, distance_threshold=0.05, curvature_threshold=0.5, use_smoothness=True, use_curvature=True):
        voxel_size = 0.01
        self.pcd = pcd.voxel_down_sample(voxel_size)
        
        # Estimate normals
        radius_normal = voxel_size * 5
        self.pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
        self.pcd.orient_normals_consistent_tangent_plane(k=10)
        self.pcd.normals = o3d.utility.Vector3dVector(-np.asarray(self.pcd.normals))
        
        self.num_neighbors = num_neighbors
        self.smoothness_threshold = smoothness_threshold
        self.distance_threshold = distance_threshold
        self.curvature_threshold = curvature_threshold
        self.use_smoothness = use_smoothness
        self.use_curvature = use_curvature
        
        self.labels = np.array([-1] * len(self.pcd.points))  # -1 indicates unlabeled
        self.normals = np.asarray(self.pcd.normals)
        self.pcd_tree = o3d.geometry.KDTreeFlann(self.pcd)

    def precompute_neighbors(self):
        neighbors = []
        for i in range(len(self.pcd.points)):
            [k, idx, _] = self.pcd_tree.search_radius_vector_3d(self.pcd.points[i], self.distance_threshold)
            neighbors.append(np.array(idx[1:]))  # Skip the first index as it's the point itself
        return neighbors

    def calculate_segmentation_criteria(self, neighbor_index, neighbors):
        neighbor_normal = self.normals[neighbor_index]
        second_order_neighbors = neighbors[neighbor_index]

        # Filter second-order neighbors to keep only those already segmented as PBR
        filtered_neighbors = [n for n in second_order_neighbors if self.labels[n] != -1]
        if len(filtered_neighbors) == 0:
            return float('inf')

        filtered_normals = self.normals[filtered_neighbors]
        dot_products = np.clip(np.dot(filtered_normals, neighbor_normal), -1.0, 1.0)

        return np.min(dot_products)

    def estimate_curvature(self, index):
        k, idx, _ = self.pcd_tree.search_radius_vector_3d(self.pcd.points[index], self.distance_threshold)
        if k > 1:
            neighbor_normals = self.normals[idx, :]
            curvature = np.mean(np.linalg.norm(np.cross(neighbor_normals - self.normals[index], neighbor_normals), axis=1))
            return curvature
        else:
            return 0

    def grow_region(self, starting_index, neighbors):
        region_index = 0
        queue = deque([starting_index])
        self.labels[starting_index] = region_index

        while queue:
            current_index = queue.popleft()
            current_normal = self.normals[current_index]

            neighbor_indices = neighbors[current_index]

            for neighbor_index in neighbor_indices:
                if self.labels[neighbor_index] != -1:
                    continue

                # Calculate segmentation criteria for the neighbor
                min_dot_product = self.calculate_segmentation_criteria(neighbor_index, neighbors)

                # Compute curvature for the neighbor
                curvature = self.estimate_curvature(neighbor_index)

                # Apply the thresholds based on user input
                if (self.use_smoothness and min_dot_product >= self.smoothness_threshold) or \
                   (self.use_curvature and curvature < self.curvature_threshold):
                    self.labels[neighbor_index] = region_index
                    queue.append(neighbor_index)

    def segment(self):
        # Compute the bounding box of the point cloud
        points = np.asarray(self.pcd.points)
        min_bound = points.min(axis=0)
        max_bound = points.max(axis=0)

        # Compute the centroid of the bounding box in the x and y dimensions
        centroid_x = (min_bound[0] + max_bound[0]) / 2.0
        centroid_y = (min_bound[1] + max_bound[1]) / 2.0

        # Find the point with the highest z value near the centroid in x and y dimensions
        distances = np.linalg.norm(points[:, :2] - np.array([centroid_x, centroid_y]), axis=1)
        highest_point_index = np.argmax(points[:, 2] - distances)

        neighbors = self.precompute_neighbors()
        self.grow_region(highest_point_index, neighbors)

        return self.pcd, self.labels

    def color_point_cloud(self):
        points = np.asarray(self.pcd.points)
        colors = np.zeros_like(points)

        # Color all points grey
        colors[:, :] = [0.5, 0.5, 0.5]  # Grey color

        # Color the labeled points red
        colors[self.labels != -1] = [1, 0, 0]  # Red color

        # Create a new point cloud with the updated colors
        colored_pcd = o3d.geometry.PointCloud()
        colored_pcd.points = o3d.utility.Vector3dVector(points)
        colored_pcd.colors = o3d.utility.Vector3dVector(colors)

        self.pcd.colors = o3d.utility.Vector3dVector(colors)

        return self.pcd

# Load your point cloud
pcd = o3d.io.read_point_cloud("box_pbr/pbr28.pcd")
segmenter = RegionGrowingSegmentation(pcd,
                                      distance_threshold=0.05,
                                      smoothness_threshold=0.95, #0.93 a lil loose
                                      curvature_threshold = 0.15,
                                      #use_curvature=False,
                                     )
pcd, labels = segmenter.segment()

# Color the point cloud
colored_pcd = segmenter.color_point_cloud()

# Visualize the point cloud
o3d.visualization.draw_geometries([colored_pcd],
                                  #point_show_normal = True,
                                 )


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


KeyboardInterrupt: 

In [231]:
import open3d as o3d
import numpy as np

def highlight_seeds(pcd, highest_point_index, bottom_most_point_index):
    points = np.asarray(pcd.points)
    colors = np.zeros_like(points)

    # Color all points in blue
    #colors[:, :] = [0, 0, 1]  # Blue color

    # Color the highest point in red
    colors[highest_point_index] = [1, 0, 0]  # Red color

    # Color the bottom-most point in green
    colors[bottom_most_point_index] = [0, 1, 0]  # Green color

    # Create a new point cloud with the updated colors
    highlighted_pcd = o3d.geometry.PointCloud()
    highlighted_pcd.points = o3d.utility.Vector3dVector(points)
    highlighted_pcd.colors = o3d.utility.Vector3dVector(colors)

    return highlighted_pcd

if __name__ == "__main__":
    # Load your point cloud
    pcd = o3d.io.read_point_cloud("box_pbr/pbr67.pcd")

    # Compute the bounding box of the point cloud
    points = np.asarray(pcd.points)
    min_bound = points.min(axis=0)
    max_bound = points.max(axis=0)

    # Compute the centroid of the bounding box in the x and y dimensions
    centroid_x = (min_bound[0] + max_bound[0]) / 2.0
    centroid_y = (min_bound[1] + max_bound[1]) / 2.0

    # Find the point with the highest z value near the centroid in x and y dimensions
    distances = np.linalg.norm(points[:, :2] - np.array([centroid_x, centroid_y]), axis=1)
    highest_point_index = np.argmax(points[:, 2] - distances)

    # Find the point in the bottom-most corner of the bounding box
    bottom_most_point_index = np.argmin(points[:, 2])

    # Highlight the seeds
    highlighted_pcd = highlight_seeds(pcd, highest_point_index, bottom_most_point_index)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([highlighted_pcd])


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [2]:
# Code to Visualize the neighbours

import open3d as o3d
import numpy as np

# Load the point cloud

pcd = o3d.io.read_point_cloud("box_pbr/pbr28.pcd")

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))
pcd.orient_normals_consistent_tangent_plane(k=10)
pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))
        
# Convert point cloud to numpy array for easy manipulation
points = np.asarray(pcd.points)

# Find the point with the maximum Z value
max_z_index = np.argmax(points[:, 2])
max_z_point = points[max_z_index]

# Define radius and max_nn for neighbor search
radius = 0.05  # Adjust as necessary
max_nn = 1   # Adjust as necessary

# Create a KDTree for neighbor search
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

# Search for neighbors within the specified radius
[k, idx, _] = pcd_tree.search_radius_vector_3d(max_z_point, radius)

# Initialize colors: blue for all points
colors = np.array([[0, 0, 1] for _ in range(len(points))])

# Set colors of the neighbors to red
colors[idx[1:], :] = [1, 0, 0]  # Exclude the point itself by idx[1:]

# Update the point cloud colors
pcd.colors = o3d.utility.Vector3dVector(colors)

# Highlight the point with the max Z value in green
colors[max_z_index] = [0, 1, 0]

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd],)
                                  #point_show_normal = True,)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
# Code to Visualize the curvature of point cloud

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def estimate_curvature(pcd):
    # Make sure normals are estimated

    voxel_size = 0.01
    radius_normal = voxel_size * 5
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
    
    # Create KDTree
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    # Calculate curvature based on variance of normals
    curvatures = []
    for i in range(len(pcd.points)):
        k, idx, _ = pcd_tree.search_radius_vector_3d(pcd.points[i], radius_normal)
        if k > 1:
            neighbor_normals = np.asarray(pcd.normals)[idx, :]
            curvature = np.mean(np.linalg.norm(np.cross(neighbor_normals - pcd.normals[i], neighbor_normals), axis=1))
            curvatures.append(curvature)
        else:
            curvatures.append(0)

    return np.array(curvatures)

def visualize_curvature(pcd, curvatures, threshold):
    # Colors: Above threshold in red, below threshold in gray
    above_color = [1, 0, 0]  # Red
    below_color = [0.5, 0.5, 0.5]  # Gray

    # Assign colors based on curvature threshold
    colors = [above_color if curvature > threshold else below_color for curvature in curvatures]
    
    # Set colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd], window_name="Curvature Threshold Visualization")


# Load your point cloud
pcd = o3d.io.read_point_cloud("../PBR_iphone.pcd")

# Estimate curvature
curvatures = estimate_curvature(pcd)

# Visualize
visualize_curvature(pcd, curvatures, threshold = 0.15)

percentiles = np.percentile(curvatures, [25, 50, 75, 90])
print(f"25th percentile: {percentiles[0]:.4f}")
print(f"50th percentile (median): {percentiles[1]:.4f}")
print(f"75th percentile: {percentiles[2]:.4f}")
print(f"90th percentile: {percentiles[3]:.4f}")


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
25th percentile: 0.0441
50th percentile (median): 0.0833
75th percentile: 0.1709
90th percentile: 0.3787


In [59]:
# Code to Visualize the planarity of point cloud

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def estimate_planarity(pcd):
    # Ensure normals are estimated
    radius_normal = 0.1  # Adjust radius for normal estimation
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    
    # Create KDTree
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    # Calculate variance of normals
    planarities = []
    for i in range(len(pcd.points)):
        k, idx, _ = pcd_tree.search_radius_vector_3d(pcd.points[i], radius_normal)
        if k > 1:
            neighbor_normals = np.asarray(pcd.normals)[idx, :]
            variance = np.var(neighbor_normals, axis=0)
            planarity = np.sum(variance)
            planarities.append(planarity)
        else:
            planarities.append(0)

    return np.array(planarities)

def visualize_planarity(pcd, planarities, threshold):
    # Colors: Below threshold in blue, above threshold in yellow
    below_color = [0, 0, 1]  # Blue
    above_color = [1, 1, 0]  # Yellow

    # Assign colors based on planarity threshold
    colors = [below_color if planarity < threshold else above_color for planarity in planarities]
    
    # Set colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd], window_name="Planarity Visualization")

# Load your point cloud
pcd = o3d.io.read_point_cloud("box_pbr/pbr28.pcd")

# Estimate planarity
planarities = estimate_planarity(pcd)

# Set a specific planarity threshold
planarity_threshold = 0.4  # Adjust this value based on your data

# Visualize with threshold
visualize_planarity(pcd, planarities, planarity_threshold)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [6]:
# Code to Visualize the smoothness of point cloud


import numpy as np
import open3d as o3d
from collections import deque
import matplotlib.cm as cm

class RegionGrowingSegmentation:
    def __init__(self, pcd, num_neighbors=50, smoothness_threshold=0.99, distance_threshold=0.05 ):
        voxel_size = 0.02
        self.pcd = pcd.voxel_down_sample(voxel_size)
        
        # Estimate normals
        radius_normal = voxel_size * 5
        self.pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
        self.pcd.orient_normals_consistent_tangent_plane(k=10)
        self.pcd.normals = o3d.utility.Vector3dVector(-np.asarray(self.pcd.normals))
        
        self.num_neighbors = num_neighbors
        self.smoothness_threshold = smoothness_threshold
        self.distance_threshold = distance_threshold
        
        self.normals = np.asarray(self.pcd.normals)
        self.pcd_tree = o3d.geometry.KDTreeFlann(self.pcd)

    def precompute_neighbors(self):
        neighbors = []
        for i in range(len(self.pcd.points)):
            [k, idx, _] = self.pcd_tree.search_radius_vector_3d(self.pcd.points[i], self.distance_threshold)
            neighbors.append(np.array(idx[1:]))  # Skip the first index as it's the point itself
        print("checkpoint 1")
        return neighbors

    def compute_smoothness(self):
        points = np.asarray(self.pcd.points)
        smoothness = np.zeros(len(points))
        neighbors = self.precompute_neighbors()
        print("checkpoint 2")

        for i in range(len(points)):
            current_normal = self.normals[i]
            neighbor_indices = neighbors[i]
            if len(neighbor_indices) == 0:
                continue  # Skip if there are no neighbors
            
            neighbor_normals = self.normals[neighbor_indices]
            second_order_neighbor_indices = np.hstack([neighbors[n_idx] for n_idx in neighbor_indices])
            if len(second_order_neighbor_indices) == 0:
                continue  # Skip if there are no second-order neighbors
            
            second_order_normals = self.normals[second_order_neighbor_indices]
            dot_products = np.clip(np.dot(second_order_normals, current_normal), -1.0, 1.0)

        
            min_dot_product = np.min(dot_products)
            smoothness[i] = min_dot_product
        print("checkpoint 3")

        return smoothness

    def visualize_smoothness(self, smoothness):
        points = np.asarray(self.pcd.points)
        colors = np.full(points.shape, [0.5, 0.5, 0.5])  # Initialize all points with grey color

        # Apply heatmap colors to points with smoothness less than the threshold
        mask = smoothness < self.smoothness_threshold
        heatmap_colors = cm.viridis((smoothness[mask] - smoothness.min()) / (smoothness.max() - smoothness.min()))[:, :3]
        colors[mask] = heatmap_colors

        colored_pcd = o3d.geometry.PointCloud()
        colored_pcd.points = o3d.utility.Vector3dVector(points)
        self.pcd.colors = o3d.utility.Vector3dVector(colors)

        return self.pcd

# Example usage
if __name__ == "__main__":
    # Load your point cloud
    pcd = o3d.io.read_point_cloud("../PBR_iphone.pcd")

    segmenter = RegionGrowingSegmentation(pcd)
    smoothness = segmenter.compute_smoothness()

    # Visualize smoothness
    colored_pcd = segmenter.visualize_smoothness(smoothness)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([colored_pcd])


checkpoint 1
checkpoint 2
checkpoint 3
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [12]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def estimate_normal_divergence(pcd, radius_normal=0.1):
    # Ensure normals are estimated
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(k=10)
    pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))
    
    # Create KDTree
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    # Calculate normal divergence based on angular difference
    divergences = []
    for i in range(len(pcd.points)):
        k, idx, _ = pcd_tree.search_radius_vector_3d(pcd.points[i], radius_normal)
        if k > 1:
            neighbor_normals = np.asarray(pcd.normals)[idx, :]
            current_normal = np.asarray(pcd.normals)[i]
            angles = np.arccos(np.clip(np.dot(neighbor_normals, current_normal), -1.0, 1.0))
            divergence = np.mean(angles)
            divergences.append(divergence)
        else:
            divergences.append(0)

    return np.array(divergences)

def visualize_normal_divergence(pcd, divergences, threshold):
    # Colors: Above threshold in red, below threshold in gray
    above_color = [1, 0, 0]  # Red
    below_color = [0.5, 0.5, 0.5]  # Gray

    # Assign colors based on divergence threshold
    colors = [above_color if divergence > threshold else below_color for divergence in divergences]
    
    # Set colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd], window_name="Normal Divergence Visualization")

# Load your point cloud
pcd = o3d.io.read_point_cloud("../box_pbr/pbr67.pcd")

# Estimate normal divergence
divergences = estimate_normal_divergence(pcd, radius_normal=0.1)

# Visualize
visualize_normal_divergence(pcd, divergences, threshold=np.deg2rad(20))  # Threshold in radians (e.g., 20 degrees)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [20]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def estimate_normal_convergence(pcd, radius_normal=0.1):
    # Ensure normals are estimated
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(k=10)
    pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))
    
    # Create KDTree
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    # Calculate normal convergence and divergence
    convergences = []
    divergences = []
    for i in range(len(pcd.points)):
        k, idx, _ = pcd_tree.search_radius_vector_3d(pcd.points[i], radius_normal)
        if k > 1:
            neighbor_normals = np.asarray(pcd.normals)[idx, :]
            current_normal = np.asarray(pcd.normals)[i]
            angles = np.arccos(np.clip(np.dot(neighbor_normals, current_normal), -1.0, 1.0))
            mean_angle = np.mean(angles)
            std_angle = np.std(angles)
            convergences.append(mean_angle)
            divergences.append(std_angle)
        else:
            convergences.append(np.pi)  # If no neighbors, set to max divergence (180 degrees)
            divergences.append(0)

    return np.array(convergences), np.array(divergences)

def visualize_normal_convergence(pcd, convergences, divergences, mean_threshold, std_threshold):
    # Colors: Converging regions in red, non-converging regions in gray
    converging_color = [1, 0, 0]  # Red for converging normals
    non_converging_color = [0.5, 0.5, 0.5]  # Gray for non-converging normals

    # Assign colors based on convergence and divergence thresholds
    colors = [
        converging_color if (convergence > mean_threshold and divergence > std_threshold) else non_converging_color
        for convergence, divergence in zip(convergences, divergences)
    ]
    
    # Set colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd], window_name="Normal Convergence Visualization")

# Load your point cloud
pcd = o3d.io.read_point_cloud("../box_pbr/pbr42.pcd")

# Estimate normal convergence and divergence
convergences, divergences = estimate_normal_convergence(pcd, radius_normal=0.1)

# Visualize
visualize_normal_convergence(pcd, convergences, divergences, mean_threshold=np.deg2rad(20), std_threshold=np.deg2rad(0))  # Adjust thresholds as needed


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def estimate_normal_divergence(pcd, radius_normal=0.1):
    # Ensure normals are estimated
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    
    # Reorient normals
    pcd.orient_normals_consistent_tangent_plane(k=10)
    pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))
    
    # Create KDTree
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    # Calculate normal divergence based on angular difference
    divergences = []
    for i in range(len(pcd.points)):
        k, idx, _ = pcd_tree.search_radius_vector_3d(pcd.points[i], radius_normal)
        if k > 1:
            neighbor_normals = np.asarray(pcd.normals)[idx, :]
            current_normal = np.asarray(pcd.normals)[i]
            angles = np.arccos(np.clip(np.dot(neighbor_normals, current_normal), -1.0, 1.0))
            std_angle = np.std(angles)
            divergences.append(std_angle)
        else:
            divergences.append(0)

    return np.array(divergences)

def visualize_normal_divergence(pcd, divergences, std_threshold):
    # Colors: Diverging regions in red, non-diverging regions in gray
    diverging_color = [1, 0, 0]  # Red for diverging normals
    non_diverging_color = [0.5, 0.5, 0.5]  # Gray for non-diverging normals

    # Assign colors based on divergence threshold
    colors = [
        diverging_color if divergence > std_threshold else non_diverging_color
        for divergence in divergences
    ]
    
    # Set colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd], window_name="Normal Divergence Visualization")

# Load your point cloud
pcd = o3d.io.read_point_cloud("../box_pbr/pbr42.pcd")

# Estimate normal divergence
divergences = estimate_normal_divergence(pcd, radius_normal=0.1)

# Visualize
visualize_normal_divergence(pcd, divergences, std_threshold=np.deg2rad(10))  # Adjust threshold as needed


In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


In [3]:
pcd = o3d.io.read_point_cloud("../box_pbr_annotation/pbr28.pcd")

In [17]:
np.shape(np.asarray(pcd.points))

(91311, 3)

In [12]:
pcd = o3d.io.read_point_cloud("../box_pbr/pbr28.pcd")

In [13]:
np.shape(np.asarray(pcd.colors))

(91311, 3)

In [23]:
import numpy as np
import open3d as o3d
import laspy

def print_original_cloud_index(las_file_path):
    # Open the LAS file
    las = laspy.read(las_file_path)
    
    # Check if the "Original cloud index" field exists
    if "Original cloud index" in las.point_format.dimension_names:
        original_cloud_index = np.int_(las["Original cloud index"])
        print("Original Cloud Index values:")
        print(original_cloud_index)
        return original_cloud_index
    else:
        print("The 'Original cloud index' field does not exist in the LAS file.")
        return None

def apply_labels_to_pcd(pcd, labels):
    # Apply labels to the point cloud as colors
    colors = np.zeros((len(labels), 3))
    colors[labels == 0] = [1, 0, 0]  # Red for label 0
    colors[labels == 1] = [0, 0, 1]  # Blue for label 1
    
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

def main():
    las_file_path = "../box_pbr_annotation/pbr28.las"  # Replace with your .las file path
    pcd_file_path = "../box_pbr/pbr28.pcd"  # Replace with your .pcd file path

    # Get the ground truth labels from the LAS file
    labels = print_original_cloud_index(las_file_path)
    if labels is None:
        return

    # Load the point cloud
    pcd = o3d.io.read_point_cloud(pcd_file_path)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Reorient normals
    pcd.orient_normals_consistent_tangent_plane(k=10)
    pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))
    print(f"Loaded point cloud with {len(pcd.points)} points.")

    # Apply the labels as colors to the point cloud
    pcd = apply_labels_to_pcd(pcd, labels)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

if __name__ == "__main__":
    main()


Original Cloud Index values:
[0 0 0 ... 1 1 1]
Loaded point cloud with 91311 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [26]:
import laspy
import numpy as np
import open3d as o3d

def load_las_as_open3d_point_cloud(las_file_path):
    # Read LAS file
    pc = laspy.read(las_file_path)
    x = pc.x.scaled_array()
    y = pc.y.scaled_array()
    z = pc.z.scaled_array()
    
    if "Original cloud index" in pc.point_format.dimension_names:
        ground_truth_labels = np.int_(pc["Original cloud index"])
    else:
        raise ValueError("The 'Original cloud index' field does not exist in the LAS file.")
    
    # Normalize coordinates
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    z_mean = np.mean(z)
    xyz = np.vstack((x - x_mean, y - y_mean, z - z_mean)).transpose()
    
    # Extract RGB colors if available
    if "red" in pc.point_format.dimension_names and "green" in pc.point_format.dimension_names and "blue" in pc.point_format.dimension_names:
        r = np.uint8(pc.red / 65535. * 255)
        g = np.uint8(pc.green / 65535. * 255)
        b = np.uint8(pc.blue / 65535. * 255)
        rgb = np.vstack((r, g, b)).transpose()
    else:
        rgb = np.zeros((len(x), 3))

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(rgb)

    return pcd, ground_truth_labels

def apply_labels_to_pcd(pcd, labels):
    # Apply labels to the point cloud as colors
    colors = np.asarray(pcd.colors)  # Preserve original colors
    colors[labels == 1] = [1, 0, 0]  # Red for label 1
    colors[labels == 0] = [0, 0, 1]  # Blue for label 0

    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

def main():
    las_file_path = "../box_pbr_annotation/pbr28.las"  # Replace with your .las file path

    # Load the LAS file as an Open3D point cloud and get ground truth labels
    pcd, ground_truth_labels = load_las_as_open3d_point_cloud(las_file_path)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    # Reorient normals
    pcd.orient_normals_consistent_tangent_plane(k=10)
    pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))
    print(f"Loaded point cloud with {len(pcd.points)} points.")

    # Apply the labels as colors to the point cloud
    pcd = apply_labels_to_pcd(pcd, ground_truth_labels)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

if __name__ == "__main__":
    main()


Loaded point cloud with 91311 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
